In [1]:
from servoserial import ServoSerial
servo = ServoSerial() # 권한 오류 -> 터미널창 -> sudo chmod 777 /dev/ttyTHS1

serial Open!


In [ ]:
# chmod -> change mode 
# 리눅스의 권한을 주는 방식
# sudo chmod 777 /dev/ttyTHS1
# ____ _____ _____ read, write, execute 
# user group other
# 111  101   001 -> sudo chmod 751

In [3]:
# 내 로봇 머리의 중앙값 찾기
# 1번 좌우 600 ~ 3600 / 2번 위아래 1300 ~ 4095
servo.Servo_serial_double_control(1, 3300, 2, 2150)

205
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0c\xe4\x00\n\x02\x08f\x00\n\xcd'


In [4]:
center_x = 3300
center_y = 2150

### PID (비례-적분-미분)
- 카메라가 얼굴 탐지 영역의 중심점을 찾아가는데 
- 어느 정도로 움직여야 할지 그 정도를 계산하는 라이브러리

In [ ]:
# P (propotional : 비례)
# 현재 물체와 원하는 위치 사이의 오차에 비례해서 움직임을 조절
# -> 오차가 크면 많이 움직이고 적으면 조금 움직임 -> 오차를 빠르게 줄여주는 역할
# I (integral : 적분)
# 오랜시간동안 물체가 원하는 위치에 도달하지 못했을 때 이전 오차들을 더한 후 누적해서 물체를 움직임
# -> 오차를 완전히 없애주고 정확한 위치로 가게 함
# D (derivative : 미분)
# 물체가 급격히 위치를 바꿀 때 이 변화를 미리 감지하고 조절
# -> 로봇이 너무 빨리 이동하지 않게 함
# --> 세 가지 요소를 결합해서 로봇머리가 사람 얼굴을 찾아 부드럽고 정확하게 따라가도록 함

In [5]:
import PID
xservo_pid = PID.PositionalPID(1.9, 0.3, 0.35)
yservo_pid = PID.PositionalPID(1.5, 0.2, 0.3)

In [6]:
# 1. 카메라 인스턴스 생성
from jetbot import Camera # 카메라 관련 라이브러리
from jetbot import bgr8_to_jpeg # bgr type -> jpeg 변환
camera = Camera.instance(width=720, height=720)

In [7]:
# 2. 카메라 프리뷰 생성
import ipywidgets.widgets as widgets # 파이썬에서 위젯을 활용할 수 있는 라이브러리
from IPython.display import display # 실제 이미지를 display할 수 있는 라이브러리
detect_face = widgets.Image(width = 300, height = 300, format = 'jpeg')
display(detect_face)

Image(value=b'', format='jpeg', height='300', width='300')

In [8]:
!curl -L -O https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml
!curl -L -O https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_eye.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  908k  100  908k    0     0   384k      0  0:00:02  0:00:02 --:--:--  470k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  333k  100  333k    0     0   369k      0 --:--:-- --:--:-- --:--:-- 1888k


In [9]:
import cv2
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_x = face_y = face_w = face_h = 0  # 얼굴을 표시할 좌표를 0으로 초기화

eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
# 눈 좌표 초기화
eye_x = xye_y = eye_w = eye_h = 0

In [ ]:
while 1:
    frame = camera.value  # 실시간으로 돌아가는 카메라 이미지
    cv2.resize(frame,(300,300)) # 학습된 데이터의 이미지가 300*300픽셀
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 흑백으로 변경
    faces = face_cascade.detectMultiScale(gray) # 흑백이미지 내에서 얼굴 검출
    if len(faces) > 0:
        (face_x,face_y,face_w,face_h) = faces[0]
        # 직사각형 그리기!
        cv2.rectangle(frame,(face_x,face_y),(face_x+face_w,face_y+face_h),(0,0,255),3)
        
        # X축
        xservo_pid.SystemOutput = (face_x+face_w)/2  # 얼굴의 x축 중심점
        xservo_pid.SetStepSignal(150)  # 단계 신호 설정 -> 얼굴까지 얼마나 빠르게 움직여야 되는가
        xservo_pid.SetInertiaTime(0.01, 0.006) # 관성시간 -> 변화에 대해서 얼마나 빠르게 대응하는가
        target_x = int(center_x + xservo_pid.SystemOutput)
        
        # Y축
        yservo_pid.SystemOutput = (face_y+face_h)/2  # 얼굴의 y축 중심점
        yservo_pid.SetStepSignal(150)  # 단계 신호 설정 -> 얼굴까지 얼마나 빠르게 움직여야 되는가
        yservo_pid.SetInertiaTime(0.01, 0.006) # 관성시간 -> 변화에 대해서 얼마나 빠르게 대응하는가
        target_y = int(center_y + yservo_pid.SystemOutput)
        
        servo.Servo_serial_double_control(1, target_x, 2, target_y)
            
        #눈이 검출된다 -> 먼저 얼굴이 검출되어야 함
        roi_region = gray[face_y: face_y+face_h, face_x : face_x+face_w]
                                    #-> gray전체 프레임에서 얼굴영역만 슬라이싱
        roi_color = frame[face_y: face_y+face_h, face_x : face_x+face_w]
                                    #-> 눈을 표시할 수 있는 도형을 컬러프레임에 찍기 위함!
        eyes = eye_cascade.detectMultiScale(roi_region)
        #눈이 검출되었다면!
        for eye_x, eye_y, eye_w, eye_h in eyes:
            cv2.rectangle(roi_color, (eye_x,eye_y), (eye_x+eye_w, eye_y+eye_h), (255,0,0), 2)

    detect_face.value = bgr8_to_jpeg(frame)  # 카메라 출력

75
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0c\xec\x00\n\x02\x07\xe1\x00\nK'
89
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0c\x91\x00\n\x02\x08-\x00\nY'
85
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0cZ\x00\n\x02\x08h\x00\nU'
93
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0c@\x00\n\x02\x08z\x00\n]'
102
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0c5\x00\n\x02\x08|\x00\nf'
112
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0c*\x00\n\x02\x08}\x00\np'
123
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0c\x1d\x00\n\x02\x08\x7f\x00\n{'
130
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0c\x13\x00\n\x02\x08\x82\x00\n\x82'
134
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0c\x0c\x00\n\x02\x08\x85\x00\n\x86'
137
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0c\x05\x00\n\x02\x08\x89\x00\n\x89'
143
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b\xfe\x00\n\x02\x08\x8b\x00\n\x8f'
148
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b\xf5\x00\n\x02\x08\x8f\x00\n\x94'
155
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b\xef\x00\n\x02\x08\x8e\x00\n\x9b'
162
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b\xe7\x00\n\x02\x08\x8f\x00\n\xa2'
215
b'\xff\xff\xfe\x0e\x83*\x0